Notebook to scrape country/language info from [Wikipedia page](https://en.wikipedia.org/wiki/List_of_countries_by_spoken_languages) add in ISO codes from [here](https://en.wikipedia.org/wiki/ISO_3166-1), join tables and write to DB

In [211]:
from bs4 import BeautifulSoup
import pandas as pd
from pandas.io import wb
import urllib2,re,sys
sys.path.append('/usr/local/lib/python2.7/dist-packages/')

In [212]:
import pymysql
from secrets import *

conn = pymysql.connect(host='localhost',user='root',passwd=PW, db='geo',charset='utf8')
cur = conn.cursor()

In [401]:
def cleanLanguage(l):
    if l=='Min/Min Nan':
        return u'Min'
    return re.sub(r'\(|\)','',l).strip()

### Grab Language by Country

In [355]:
wiki = "https://en.wikipedia.org/wiki/List_of_countries_by_spoken_languages"
header = {'User-Agent': 'Mozilla/5.0'} #Needed to prevent 403 error on Wikipedia
req = urllib2.Request(wiki,headers=header)
page = urllib2.urlopen(req)
soup = BeautifulSoup(page)
 
headings=soup.findAll("h3")

In [484]:
tableData={}
tables = soup.findAll("table", { "class" : "wikitable"})
tables=tables[0:]

soFar=set()

for n,table in enumerate(tables): 
    # First few tables not languages
#    print n
    lang=table.findPrevious('h3').text.partition('[')[0]
    if lang in soFar:
        lang=table.findPrevious('h4').text.partition('[')[0]
    if lang in soFar:
        lang=table.findPrevious('h2').text.partition('[')[0]
    # Some language headings are h4 not h3
    
    if lang==u'Min/Min Nan':
        lang='Min'
    
    
    soFar.add(lang)
    print '+++Adding',lang
    
    rows=[r for r in table.findAll('tr')[0].text.split('\n') if len(r)>0]
#    print rows
#   print ''
    
    data=[[] for r in rows]
    
    for n,row in enumerate(table.findAll('tr')[1:]):
                
        vals=[r for r in row.text.split('\n')]
#        print vals
        if len(vals[-1])==0:
            vals.pop(-1)
        if len(vals[0])==0 and n==0:
            vals.pop(0)
        if len(vals)>len(rows):
            vals.pop(0)
        
        if n==0:
            fillValue=vals[0]
            fillValue=cleanLanguage(fillValue)
#            rows[0]=fillValue
#            print fillValue,lang

#        print rows
#        print '=>'
#        print vals
#        print ''
        
        if u'Regions' in rows:
#            print 'Setting fillvalue to',lang,vals[0],fillValue
            vals[0]=fillValue
            
#            print vals
        
        if rows[0]=='Regions':
            vals[0]=lang
#            print vals
            
            
        if len(vals)==len(rows):
            for nn,val in enumerate(vals):
                data[nn].append(val)
        else:
            print 'Length error : ',vals

        try:
            if len(data[0][-1])==0:
                data[0][-1]=data[0][-2]
                # Fill forward
        except:
            print 'Cant fill forward',vals
#    print '--------------------------'
    tableData[lang]=(pd.DataFrame(data=dict(zip(rows,data))))
#    print ''

+++Adding Albanian
+++Adding Armenian
+++Adding Baltic
+++Adding Celtic
+++Adding Germanic
Length error :  [u'Low German', u'(Low Saxon)', u'\xa0Germany', u'Niederdeutsch, recognized as regional language']
+++Adding Dutch
+++Adding English
Length error :  [u'English', u'\xa0FS. Micronesia']
Length error :  [u'English', u'Dependent entities']
+++Adding German
+++Adding Greek
+++Adding Iranian
+++Adding Indo-Aryan
+++Adding Domari
+++Adding Romani
+++Adding Other Indo-Aryan
+++Adding Italic
+++Adding Italian
+++Adding Portuguese
+++Adding Spanish
+++Adding Other Italic Lang.
+++Adding Slavic
Length error :  [u'Russian', u'', u'See also: List of countries where Russian is an official language', u'']
+++Adding Mongolic languages
+++Adding Turkic languages
+++Adding Arabic
+++Adding Other Semitic Lang.
+++Adding Dravidian languages
+++Adding Japonic languages
+++Adding Language isolate
Cant fill forward [u'', u'See also: Basque Country (historical territory)', u'']
+++Adding South Caucasian

In [485]:
tableData['Baltic']

,Country,Language,Status
0,Canada,Baltic,"unofficial, spoken in Canada"
1,Latvia,Latgalian,"unofficial, spoken in Latgale"
2,Russia,Latgalian,significant minority in Siberia
3,Latvia,Latvian,official
4,Latvia,Lithuanian,minority
5,Lithuania,Lithuanian,official
6,Poland,Lithuanian,"minority, majority in the city of Puńsk"


In [486]:
for k,v in tableData.items():
    print k
#    print v.columns
    v['name']=v['Country'].apply(unicode.strip)
    del v['Country']
    
    v.index=v.name
    
    if 'Regions' in v.columns:
        print '\t',k
        v['Language']=k
        v['type']=v['Regions']
        del v['Regions']
    else:
        v['type']=k
    

Bornean
Min
	Min
Cherokee
Celtic
East Fijian
Mandarin
	Mandarin
Italic
	Italic
Dutch
	Dutch
Other Indo-Aryan
Dravidian languages
South Caucasian languages
	South Caucasian languages
Other Sino-Tibetan
Slavic
Uralic languages
Malayo-Sumbawan
Romani
	Romani
Tai–Kadai
Mongolic languages
Domari
	Domari
Armenian
	Armenian
Polynesian
Apachean
Indo-Aryan
Philippines
Other Italic Lang.
Hmong–Mien
Germanic
Iranian
Language isolate
Albanian
Cantonese
	Cantonese
Yue Chinese
	Yue Chinese
Greek
English
	English
Baltic
Sunda–Sulawesi
Standard Chinese
Italian
	Italian
Portuguese
Japonic languages
Hakka
	Hakka
Turkic languages
German
	German
Micronesian
Other Semitic Lang.
Other Sinitic Lang.
Spanish
Arabic
	Arabic


In [487]:
tableData['Turkic languages']

,Language,Status,name,type
name,,,,
Azerbaijan,Azerbaijani,official,Azerbaijan,Turkic languages
Iran,Azerbaijani,co-official in Iranian Azerbaijan,Iran,Turkic languages
Iraq,Azerbaijani,minority,Iraq,Turkic languages
Georgia,Azerbaijani,regional,Georgia,Turkic languages
Russia,Azerbaijani,co-official in Dagestan,Russia,Turkic languages
Syria,Azerbaijani,minority,Syria,Turkic languages
Turkey,Azerbaijani,regional,Turkey,Turkic languages
Ukraine,Azerbaijani,minority,Ukraine,Turkic languages
Belarus,Azerbaijani,minority,Belarus,Turkic languages


In [433]:
iso.head()

,id,iso,iso2,iso3,name
name,,,,,
Afghanistan,004,ISO 3166-2:AF,AF,AFG,Afghanistan
Aland Islands !Åland Islands,248,ISO 3166-2:AX,AX,ALA,Aland Islands !Åland Islands
Albania,008,ISO 3166-2:AL,AL,ALB,Albania
Algeria,012,ISO 3166-2:DZ,DZ,DZA,Algeria
American Samoa,016,ISO 3166-2:AS,AS,ASM,American Samoa


### Grab ISO Codes/Country Names
[here](https://en.wikipedia.org/wiki/ISO_3166-1)

In [341]:
wiki = "https://en.wikipedia.org/wiki/ISO_3166-1"
header = {'User-Agent': 'Mozilla/5.0'} #Needed to prevent 403 error on Wikipedia
req = urllib2.Request(wiki,headers=header)
page = urllib2.urlopen(req)
soup = BeautifulSoup(page)

headings=soup.findAll("h3")


In [342]:
#tableData={}
tables = soup.findAll("table", { "class" : "wikitable"})
tables=tables[0:]

In [504]:
data=[]
rows=['name','iso2','iso3','id','iso']
data=[[] for k in rows]

for r in tables[0].findAll('tr'):
    for n,d in enumerate(r.findAll('td')):
#        print n,d.text
        data[n].append(d.text)
#    print ''

In [344]:
iso=pd.DataFrame(data=dict(zip(rows,data)))

In [345]:
iso.head()

,id,iso,iso2,iso3,name
0,004,ISO 3166-2:AF,AF,AFG,Afghanistan
1,248,ISO 3166-2:AX,AX,ALA,Aland Islands !Åland Islands
2,008,ISO 3166-2:AL,AL,ALB,Albania
3,012,ISO 3166-2:DZ,DZ,DZA,Algeria
4,016,ISO 3166-2:AS,AS,ASM,American Samoa


In [215]:
res=cur.execute('create table iso (name varchar(100) character set utf8,iso2 varchar(2),iso3 varchar(3))',)
# Create table of countries

In [218]:
cmd="INSERT INTO iso (name,iso2,iso3) VALUES (%s,%s,%s)"

for row in iso.iterrows():
    temp=(row[1][4],row[1][2],row[1][3])
    res=cur.execute(cmd,temp)
    conn.commit()

In [346]:
iso.head()

,id,iso,iso2,iso3,name
0,004,ISO 3166-2:AF,AF,AFG,Afghanistan
1,248,ISO 3166-2:AX,AX,ALA,Aland Islands !Åland Islands
2,008,ISO 3166-2:AL,AL,ALB,Albania
3,012,ISO 3166-2:DZ,DZ,DZA,Algeria
4,016,ISO 3166-2:AS,AS,ASM,American Samoa


In [347]:
iso.index=iso.name
# Useful for joining

In [348]:
iso.head()

,id,iso,iso2,iso3,name
name,,,,,
Afghanistan,004,ISO 3166-2:AF,AF,AFG,Afghanistan
Aland Islands !Åland Islands,248,ISO 3166-2:AX,AX,ALA,Aland Islands !Åland Islands
Albania,008,ISO 3166-2:AL,AL,ALB,Albania
Algeria,012,ISO 3166-2:DZ,DZ,DZA,Algeria
American Samoa,016,ISO 3166-2:AS,AS,ASM,American Samoa


### Join languages and countries

In [488]:
isoLangHash={}
langIsoHash={}
nameIsoHash={}

for row in iso.iterrows():
    isoLangHash[row[1][2]]=row[1][4]
    langIsoHash[row[1][4]]=row[1][2]
    nameIsoHash[row[1][4]]=row[1][2]

In [489]:
v.head()

,Status,name,Language,type
name,,,,
Islam,Liturgical language of Islam,Islam,Arabic,Arabic
Algeria,official,Algeria,Arabic,Arabic
Comoros,co-official with French,Comoros,Arabic,Arabic
Chad,co-official with French,Chad,Arabic,Arabic
Djibouti,co-official with French,Djibouti,Arabic,Arabic


In [495]:
for n,t in enumerate(tableData.values()):
    print t.columns
    t['iso']=t['name'].apply(lambda x:nameIsoHash.get(x))

Index([u'Language', u'Status', u'name', u'type', u'iso'], dtype='object')
Index([u'Status', u'name', u'Language', u'type', u'iso'], dtype='object')
Index([u'Language', u'Status', u'name', u'type', u'iso'], dtype='object')
Index([u'Language', u'Status', u'name', u'type', u'iso'], dtype='object')
Index([u'Language', u'Status', u'name', u'type', u'iso'], dtype='object')
Index([u'Status', u'name', u'Language', u'type', u'iso'], dtype='object')
Index([u'Status', u'name', u'Language', u'type', u'iso'], dtype='object')
Index([u'Status', u'name', u'Language', u'type', u'iso'], dtype='object')
Index([u'Language', u'Status', u'name', u'type', u'iso'], dtype='object')
Index([u'Language', u'Status', u'name', u'type', u'iso'], dtype='object')
Index([u'Status', u'name', u'Language', u'type', u'iso'], dtype='object')
Index([u'Language', u'Status', u'name', u'type', u'iso'], dtype='object')
Index([u'Language', u'Status', u'name', u'type', u'iso'], dtype='object')
Index([u'Language', u'Status', u'name'

In [496]:
combined=pd.concat(tableData.values())
# THis is final dataframe with all languages, iso codes and names

In [497]:
combined.head()

,Language,Status,iso,name,type
name,,,,,
Brunei,Brunei Malay,regional,BN,Brunei,Bornean
Malaysia,Brunei Malay,minority,MY,Malaysia,Bornean
Malaysia,Coastal Kadazan,minority,MY,Malaysia,Bornean
Malaysia,Iban,regional,MY,Malaysia,Bornean
Brunei,Iban,minority,BN,Brunei,Bornean


Manually add additions to iso dictionary

In [498]:
nameIsoHash['United States']='US'
nameIsoHash['United Kingdom']='GB'
nameIsoHash['Russia']='RU'
nameIsoHash['Iran']='IR'
nameIsoHash['Vietnam']='VN'
nameIsoHash['Burma']='MM'
nameIsoHash['Laos']='LA'
nameIsoHash['Brunei']='BN'
nameIsoHash['Syria']='SY'
nameIsoHash['FS. Micronesia']='FM'
nameIsoHash['Taiwan']='TW'
nameIsoHash['Macau']='MO'
nameIsoHash['Cape Verde']='CV'
nameIsoHash['Macedonia']='MK'
nameIsoHash['South Korea']='KR'
nameIsoHash['North Korea']='KP'
nameIsoHash['Moldova']='MD'
nameIsoHash['Hawaii']='US'
nameIsoHash['Tanzania']='TZ'
nameIsoHash['Palestine']='PS'
nameIsoHash['Venezuela']='VE'

In [499]:
combined[pd.isnull(combined.iso)].name.value_counts()

Kosovo                                             5
Vatican City                                       3
Islam                                              1
Canary Islands                                     1
Ivory Coast                                        1
British Virgin Islands                             1
Ceuta                                              1
Easter Island                                      1
Melilla                                            1
Scotland                                           1
Abkhazia                                           1
U.S. Virgin Islands                                1
Saint Barthélemy                                   1
Judaism                                            1
Gabon, Cocobeach                                   1
Sark                                               1
São Tomé and Príncipe                              1
Falkland Islands                                   1
Crimea  Ukraine                               

Put languages and countries in table

In [500]:
combined.head()

,Language,Status,iso,name,type
name,,,,,
Brunei,Brunei Malay,regional,BN,Brunei,Bornean
Malaysia,Brunei Malay,minority,MY,Malaysia,Bornean
Malaysia,Coastal Kadazan,minority,MY,Malaysia,Bornean
Malaysia,Iban,regional,MY,Malaysia,Bornean
Brunei,Iban,minority,BN,Brunei,Bornean


In [501]:
res=cur.execute('create table language (language varchar(100) character set utf8, country_name varchar(100) character set utf8, iso2 varchar(2),status varchar(50))',)

In [502]:
cmd="INSERT INTO language (language,country_name,iso2,status) VALUES (%s,%s,%s,%s)"

        
for row in combined.iterrows():
    print row[0],row[1].values
    temp=(row[1]['Language'],row[1]['name'],row[1]['iso'],row[1]['Status'])
    res=cur.execute(cmd,temp)
    conn.commit()


Brunei [u'Brunei Malay' u'regional' 'BN' u'Brunei' u'Bornean']
Malaysia [u'Brunei Malay' u'minority' u'MY' u'Malaysia' u'Bornean']
Malaysia [u'Coastal Kadazan' u'minority' u'MY' u'Malaysia' u'Bornean']
Malaysia [u'Iban' u'regional' u'MY' u'Malaysia' u'Bornean']
Brunei [u'Iban' u'minority' 'BN' u'Brunei' u'Bornean']
Indonesia [u'Iban' u'minority' u'ID' u'Indonesia' u'Bornean']
Comoros [u'Malagasy' u'minority' u'KM' u'Comoros' u'Bornean']
Madagascar [u'Malagasy' u'official' u'MG' u'Madagascar' u'Bornean']
Mayotte [u'Malagasy' u'minority' u'YT' u'Mayotte' u'Bornean']
Angola ['Min' u'minority' u'AO' u'Angola' 'Min']
Cape Verde ['Min' u'minority' 'CV' u'Cape Verde' 'Min']
Mauritius ['Min' u'minority' u'MU' u'Mauritius' 'Min']
South Africa ['Min' u'minority' u'ZA' u'South Africa' 'Min']
Brunei ['Min' u'minority' 'BN' u'Brunei' 'Min']
Cambodia ['Min' u'minority' u'KH' u'Cambodia' 'Min']
China ['Min' u'regional' u'CN' u'China' 'Min']
Hong Kong ['Min' u'minority' u'HK' u'Hong Kong' 'Min']
Indon

In [385]:
combined[combined.name=='Angola']

,Language,Status,iso,name
name,,,,
Angola,NaN,minority,AO,Angola
Angola,NaN,minority,AO,Angola
Angola,NaN,minority,AO,Angola


In [308]:
tableData.keys()

[u'Bornean',
 u'Cherokee',
 u'Celtic',
 u'East Fijian',
 u'Italic',
 u'Mandarin',
 u'Yue Chinese',
 u'Slavic',
 u'Malayo-Sumbawan',
 u'Other Sino-Tibetan',
 u'Armenian',
 u'Polynesian',
 u'Apachean',
 u'Min/Min Nan',
 u'Indo-Aryan',
 u'Philippines',
 u'Germanic',
 u'Iranian',
 u'Albanian',
 u'Greek',
 u'Baltic',
 u'Sunda\u2013Sulawesi',
 u'Hakka',
 u'Micronesian',
 u'Other Semitic Lang.',
 u'Other Sinitic Lang.',
 u'Arabic']

In [396]:
tableData['Arabic']

,Status,name,iso
name,,,
Islam,Liturgical language of Islam,Islam,None
Algeria,official,Algeria,DZ
Comoros,co-official with French,Comoros,KM
Chad,co-official with French,Chad,TD
Djibouti,co-official with French,Djibouti,DJ
Egypt,official,Egypt,EG
Eritrea,co-official with Tigrinya,Eritrea,ER
Libya,official,Libya,LY
Mauritania,official,Mauritania,MR


In [394]:
combined[pd.isnull(combined.Language)]

,Language,Status,iso,name
name,,,,
Angola,NaN,minority,AO,Angola
Cape Verde,NaN,minority,CV,Cape Verde
Mauritius,NaN,minority,MU,Mauritius
South Africa,NaN,minority,ZA,South Africa
Brunei,NaN,minority,BN,Brunei
Cambodia,NaN,minority,KH,Cambodia
China,NaN,regional,CN,China
Hong Kong,NaN,minority,HK,Hong Kong
Indonesia,NaN,minority,ID,Indonesia
